# Benchmark

Let's benchmark against all tool usage tasks. 

Expand the models list to benchmark with different models.

In [1]:
import datetime

from langsmith.client import Client

from langchain_benchmarks import (
    __version__,
    clone_public_dataset,
    model_registry,
    registry,
)
from langchain_benchmarks.rate_limiting import RateLimiter
from langchain_benchmarks.tool_usage.agents import (
    AnthropicToolUserFactory,
    CustomAgentFactory,
    OpenAIAgentFactory,
)

Prior to starting the tests, you may want to verify
that the task that you're working with and the models are propelry defined.

In [2]:
task = registry['Multiverse Math']

In [3]:
agent_factory = OpenAIAgentFactory(task, model='mistral-7b-instruct-v0.1') # Follows OpenAI function format
# agent_factory = OpenAIAgentFactory(task, model='gpt-3.5-turbo-1106')
agent_factory().invoke({"question": "(2 + 5) and then to the power of 0.5"})

{'input': '(2 + 5) and then to the power of 0.5',
 'output': 'The answer is 192.54605765894036.',
 'intermediate_steps': [(OpenAIToolAgentAction(tool='add', tool_input={'a': 2, 'b': 5}, log="\nInvoking: `add` with `{'a': 2, 'b': 5}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8149a232b2db4b0c92d8c80576f0047a', 'function': {'arguments': '{"a": 2, "b": 5}', 'name': 'add'}, 'type': 'function'}]})], tool_call_id='call_8149a232b2db4b0c92d8c80576f0047a'),
   8.2),
  (OpenAIToolAgentAction(tool='power', tool_input={'a': 8.2, 'b': 0.5}, log="\nInvoking: `power` with `{'a': 8.2, 'b': 0.5}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_f6a87225d4bd45348e60cbbe622495cf', 'function': {'arguments': '{"a": 8.2, "b": 0.5}', 'name': 'power'}, 'type': 'function'}]})], tool_call_id='call_f6a87225d4bd45348e60cbbe622495cf'),
   192.54605765894036)]}

Let's make an experiment id

In [4]:
experiment_uuid = "woof"  # Or generate ranom using uuid.uuid4().hex[:4]

Define the test cases

In [5]:
tests = [
    # 2-tuple of (architecture, model name)
    ("anthropic_tool_user", "claude-2.1"),
    ("openai_functions", "mistral-7b-instruct-v0.1"),
    ("openai_functions", "gpt-3.5-turbo-1106"),
    ("openai_functions", "gpt-3.5-turbo-0613"),
    ("openai_functions", "gpt-4-1106-preview"),
    ("openai_functions", "gpt-4-0613"),
]

## Run

In [7]:
import os

In [6]:
client = Client()  # Launch langsmith client for cloning datasets
today = datetime.date.today().isoformat()
rate_limiter = RateLimiter(requests_per_second=2)

for task in registry:
    if task.type != "ToolUsageTask":
        continue

    dataset_name = task.name
    clone_public_dataset(task.dataset_id, dataset_name=dataset_name)

    for arch, model in tests:
        print()
        print(f"Benchmarking {task.name} with model: {model} and arch: {arch}")
        eval_config = task.get_eval_config()

        if arch == "openai_functions":
            agent_factory = OpenAIAgentFactory(
                task, model=model, rate_limiter=rate_limiter
            )
        elif arch == "custom_agent":
            agent_factory = CustomAgentFactory(
                task, model=model, rate_limiter=rate_limiter
            )
        elif arch == "anthropic_tool_user":
            agent_factory = AnthropicToolUserFactory(task)
        else:
            raise ValueError()

        client.run_on_dataset(
            dataset_name=dataset_name,
            llm_or_chain_factory=agent_factory,
            evaluation=eval_config,
            verbose=False,
            project_name=f"{model}-{task.name}-{today}-{experiment_uuid}",
            tags=[model],
            concurrency_level=5,
            project_metadata={
                "model": model,
                "id": experiment_uuid,
                "task": task.name,
                "date": today,
                "langchain_benchmarks_version": __version__,
                "arch": arch,
            },
        )

Dataset Tool Usage - Typewriter (1 tool) already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/82ca6840-cf23-4bb0-a9be-55237ebbe9d3.

Benchmarking Tool Usage - Typewriter (1 tool) with model: claude-2.1 and arch: anthropic_tool_user
View the evaluation results for project 'claude-2.1-Tool Usage - Typewriter (1 tool)-2023-12-15-woof' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/82ca6840-cf23-4bb0-a9be-55237ebbe9d3/compare?selectedSessions=2d749062-a6a5-474d-99ce-60f6c847375c

View all tests for Dataset Tool Usage - Typewriter (1 tool) at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/82ca6840-cf23-4bb0-a9be-55237ebbe9d3
[------------------------------------------------->] 20/20
Benchmarking Tool Usage - Typewriter (1 tool) with model: mistral-7b-instruct-v0.1 and arch: openai_functions
View the evaluation results for project 'mistral-7b-instruct-

Chain failed for example 8a2b5450-dd16-4213-8b70-cb2583d6c7eb with inputs {'question': 'student'}
Error Type: BadRequestError, Message: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_EXTKZSoSqyJufBYv64TAm018", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}


[------------------------------------------------->] 20/20
Benchmarking Tool Usage - Typewriter (1 tool) with model: gpt-4-1106-preview and arch: openai_functions
View the evaluation results for project 'gpt-4-1106-preview-Tool Usage - Typewriter (1 tool)-2023-12-15-woof' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/82ca6840-cf23-4bb0-a9be-55237ebbe9d3/compare?selectedSessions=81e16b5e-ab5b-4886-86ec-c08c3d6b92fb

View all tests for Dataset Tool Usage - Typewriter (1 tool) at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/82ca6840-cf23-4bb0-a9be-55237ebbe9d3
[------->                                          ] 3/20

Chain failed for example c17e9d5a-b9f8-43dc-b5a9-6e45d21c9a2a with inputs {'question': 'communication'}
Error Type: BadRequestError, Message: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_YaB1u6onJCRe3TBblCZaVxkA", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}


[------------------------------------------------->] 20/20
Benchmarking Tool Usage - Typewriter (1 tool) with model: gpt-4-0613 and arch: openai_functions
View the evaluation results for project 'gpt-4-0613-Tool Usage - Typewriter (1 tool)-2023-12-15-woof' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/82ca6840-cf23-4bb0-a9be-55237ebbe9d3/compare?selectedSessions=5ca2a98a-8b30-4f90-97dd-9b2eefa9a591

View all tests for Dataset Tool Usage - Typewriter (1 tool) at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/82ca6840-cf23-4bb0-a9be-55237ebbe9d3
[>                                                 ] 0/20

...

[------------------------------------------------->] 21/21
Benchmarking Tool Usage - Relational Data with model: mistral-7b-instruct-v0.1 and arch: openai_functions
View the evaluation results for project 'mistral-7b-instruct-v0.1-Tool Usage - Relational Data-2023-12-15-woof' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/df6be6c9-05b3-445e-8836-ebb4aba63826/compare?selectedSessions=a60f355a-94c6-47e9-8a14-9a26a465de28

View all tests for Dataset Tool Usage - Relational Data at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/df6be6c9-05b3-445e-8836-ebb4aba63826
[------------------------------------------------->] 21/21
Benchmarking Tool Usage - Relational Data with model: gpt-3.5-turbo-1106 and arch: openai_functions
View the evaluation results for project 'gpt-3.5-turbo-1106-Tool Usage - Relational Data-2023-12-15-woof' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/df6be6c9-05b3-445e-8836-e

Chain failed for example 2a20a13d-050e-4a16-84ff-22d9582f1449 with inputs {'question': 'after calculating the sin of 1.5 radians, divide the result by cos of 1.5 radians'}
Error Type: ZeroDivisionError, Message: float division by zero


[------------------------------------------------->] 10/10
Benchmarking Multiverse Math with model: gpt-3.5-turbo-1106 and arch: openai_functions
View the evaluation results for project 'gpt-3.5-turbo-1106-Multiverse Math-2023-12-15-woof' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/108bdc68-1808-4b60-92ef-fbd9bd7e1ad0/compare?selectedSessions=91df1a9b-d6d3-44aa-99c6-6e98401f3ac7

View all tests for Dataset Multiverse Math at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/108bdc68-1808-4b60-92ef-fbd9bd7e1ad0
[------------------------------------------------->] 10/10
Benchmarking Multiverse Math with model: gpt-3.5-turbo-0613 and arch: openai_functions
View the evaluation results for project 'gpt-3.5-turbo-0613-Multiverse Math-2023-12-15-woof' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/108bdc68-1808-4b60-92ef-fbd9bd7e1ad0/compare?selectedSessions=b934ef76-8ad1-4b71-bae0-b95d4a1ef7d5

Vi

Chain failed for example 67867526-791a-452f-b534-ef2c1f5efd20 with inputs {'question': 'ecoli divides every 20 minutes. How many cells will be there after 2 hours if we start with 5 cells?'}
Error Type: InternalServerError, Message: Error code: 500 - {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID 28a9d136aed7faacc59b14d4351430dd in your email.)', 'type': 'server_error', 'param': None, 'code': None}}


[------------------------------------------------->] 10/10
Benchmarking Multiverse Math with model: gpt-4-0613 and arch: openai_functions
View the evaluation results for project 'gpt-4-0613-Multiverse Math-2023-12-15-woof' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/108bdc68-1808-4b60-92ef-fbd9bd7e1ad0/compare?selectedSessions=ea96a024-5987-47d6-9650-832bbbd4aa67

View all tests for Dataset Multiverse Math at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/108bdc68-1808-4b60-92ef-fbd9bd7e1ad0
[------------------------------------------------->] 10/10

## Inspect

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from langsmith.client import Client

Let's fetch all the data that has the same experiment ID and place it in a dataframe.

In [ ]:
experiment_ids = ["3f3e"]


def _endswith(s, suffixes):
    return any(s.endswith(suffix) for suffix in suffixes)


client = Client()
projects = [
    project
    for project in client.list_projects()
    if _endswith(project.name, experiment_ids)
]

dfs = []
for project in projects:
    # Temporary way to get tag information
    project_info = client.read_project(project_id=project.id)
    try:
        test_results = client.get_test_results(project_name=project.name)
    except Exception:
        continue

    for k, v in project_info.extra["metadata"].items():
        test_results[k] = v

    dfs.append(test_results)


df = pd.concat(dfs)

Compute a standardized "correct" column. It uses "Correct Final State" for tool usage tasks, and "correctness (which is based on output) for the other tasks.

In [ ]:
correct = []

for r in df.to_dict(orient="records"):
    if "Typewriter" in r["task"]:
        correct.append(r["feedback.Correct Final State"])
    else:
        correct.append(r["feedback.correctness"])

df["correct"] = correct
df["correct"].fillna(0, inplace=True)

Compute some statistics. We're using estimating standard error of the mean assuming a bernoulli process.

In [ ]:
num_correct = df.groupby(["model", "task"])["correct"].sum().to_frame("num_correct")
total = df.groupby(["task", "model"]).size().to_frame("total")
stats_df = total.join(num_correct)
stats_df["% correct"] = stats_df["num_correct"] / stats_df["total"]
stats_df["error"] = np.sqrt(
    stats_df["% correct"] * (1 - stats_df["% correct"]) / stats_df["total"]
)

# stats_df

models = [
    "llama-v2-70b-chat-fw",
    "mixtral-8x7b-instruct-fw",
    "claude-2",
    "claude-2.1",
    "gpt-3.5-turbo-0613",
    "gpt-3.5-turbo-1106",
    "gpt-4-0613",
    "gpt-4-1106-preview",
]

tasks = [
    "Tool Usage - Typewriter (1 tool)",
    "Tool Usage - Typewriter (26 tools)",
    "Multiverse Math",
    "Tool Usage - Relational Data",
]

stats_df = stats_df.reset_index()
stats_df = stats_df[stats_df["model"].isin(models)]

Plot the result

In [ ]:
x = np.arange(len(tasks))  # the label locations
width = 0.08  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout="constrained", figsize=(16, 4))
colormap = plt.get_cmap("Set2").colors

for idx, model in enumerate(models):
    try:
        results = stats_df.set_index("model").loc[model]
    except:
        continue

    color = colormap[idx]

    results = results.set_index("task").loc[tasks]
    measurement = results["% correct"]

    values = [round(m, 2) for m in measurement]

    offset = width * multiplier * 1.4
    rects = ax.bar(
        x + offset, values, width, label=model, yerr=results["error"], color=color
    )
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel("% Questions Answered Correctly")
ax.set_title("Tool Usage Performance")
ax.set_xticks(x + width + 0.3, tasks)
ax.legend(
    loc="center left", ncols=1, bbox_to_anchor=(1.0, 0.5), frameon=False, title="Model"
)
ax.set_ylim(0, 1.10)

plt.show()